In [ ]:
import numpy as np 
from scipy.ndimage import gaussian_filter, zoom, sobel, maximum_filter, laplace, gaussian_laplace
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import cv2
from scipy.optimize import least_squares


Just For Fun i tried to implement the feature detection method as same as used in the Pizzaro paper i.e. harris interest points detector at multiscale this multiscale was created by decomposing each image into 5 levels with sigma value 1.6 and any level further then 5 was too blur, first the initial image from the pyramid in the previous scale is blurred then zoomed in with the factors of 2 and the gaussian blurr was used the same across all the levels as the image was already downsampled at each level, i have attached the outcomes of the pyramids in the Pyramids Folder as this algorithm was working but the computational time was too high it took me more then 2 hours to finish running this algorithm since there are a lots of loops in the code i didnt had  much time to optimize it for computational efficiency so i shifted from this feature finding method to SIFT algorithm but i would still like to know how can i improve this code below for it to work 

In [ ]:
# #Iam Following the Paper's method so here iam decomposing each image into 5 levels with sigma value 1.6 and any level further then 5 was too blur 
# #first the initial image from the pyramid in the previous scale is blurred then zoomed in with the factors of 2,
# img_1 = mpimg.imread("C:/Users/shalh/OneDrive/Desktop/Underwater Image/1.tif")
# img_2 = mpimg.imread("C:/Users/shalh/OneDrive/Desktop/Underwater Image/2.tif")
# img_3 = mpimg.imread("C:/Users/shalh/OneDrive/Desktop/Underwater Image/3.tif")
# img_4 = mpimg.imread("C:/Users/shalh/OneDrive/Desktop/Underwater Image/4.tif")
# img_5 = mpimg.imread("C:/Users/shalh/OneDrive/Desktop/Underwater Image/5.tif")
# img_6 = mpimg.imread("C:/Users/shalh/OneDrive/Desktop/Underwater Image/6.tif")

# levels = 5

# def Build_pyramid(image, sigma = 1.6):
#     pyramid = [image.copy()]
#     for level in range(1,levels):
#         blurred = gaussian_filter(pyramid[-1], sigma = sigma)
#         downsampled = zoom(blurred, 0.5, order=1)
#         pyramid.append(downsampled)
#     return pyramid 


# Pyramid_im1 = Build_pyramid(img_1)
# Pyramid_im2 = Build_pyramid(img_2)
# Pyramid_im3 = Build_pyramid(img_3)
# Pyramid_im4 = Build_pyramid(img_4)
# Pyramid_im5 = Build_pyramid(img_5)
# Pyramid_im6 = Build_pyramid(img_6)

# Pyramids = [Pyramid_im1,Pyramid_im2,Pyramid_im3,Pyramid_im4,Pyramid_im5,Pyramid_im6]

In [ ]:
#Only for plotting and checking how the levels look, all the level plots for all the 6 images is in the folder Pyramids Please take a look at it 

# plt.figure(figsize=(25, 10))
# for i in range(5):
#     plt.subplot(2, 3, i+1) 
#     plt.imshow(Pyramid_im6[i], cmap='gray')  
#     plt.title(f"Level {i}")
# plt.tight_layout() 
# plt.show()



In [ ]:
# #Developing the Harris interest point algo and deploying it on all the scales in this pyramid 
# def Harris_interest_points(image, sigma_d=1.1, sigma_I=1.3):
#     smoothed = gaussian_filter(image, sigma=sigma_d, mode='constant')
#     lx = sobel(smoothed, axis=1, mode='constant')
#     ly = sobel(smoothed, axis=0, mode='constant')
#     lxy = lx*ly
#     Ix2_smoothed = gaussian_filter(lx, sigma=sigma_I, mode='constant')
#     Iy2_smoothed = gaussian_filter(ly, sigma=sigma_I, mode='constant')
#     Ixy_smoothed = gaussian_filter(lxy, sigma=sigma_I, mode='constant')
#     meu_11 = sigma_d * Ix2_smoothed
#     meu_12 = sigma_d * Ixy_smoothed
#     meu_21 = sigma_d * Ixy_smoothed
#     meu_22 = sigma_d * Iy2_smoothed
#     height,width = image.shape
#     eigenvalues = np.zeros((height,width,2))
#     for i in range(height):
#         for j in range(width):
#             M = np.array([[meu_11[i,j], meu_12[i,j]],
#                           [meu_21[i,j], meu_22[i,j]]])
#             eig_val = np.linalg.eigvals(M)
#             eigenvalues[i,j] = np.sort(eig_val)
#     return eigenvalues

# tentative_features = []

# for PY_index ,pyramid in enumerate(Pyramids):
#   for level,image in enumerate(pyramid):
#      height,width = image.shape
#      eigenvalues = Harris_interest_points(image)
#      for i in range(height):
#         for j in range(width):
#             if eigenvalues[i,j,0] > 0 and eigenvalues[i,j,1] > 0:
#                 tentative_features.append((i,j,level,PY_index))

# print(tentative_features[0])

In [ ]:

# def scale_normalized_Laplacian(image, thresh, image_num, sigma=1.4,):
#     interest_points = []
#     for index,value in enumerate(tentative_features):
#         i,j,l,py = value
        
#         if py == image_num:
#             response = []
#             for level,im in enumerate(image):
#                 print("here in loop2")
#                 if l == level:
#                   i_n,j_n = i,j
#                 elif l>level:
#                     factor = 2**(l-level)
#                     i_n,j_n = i*factor,j*factor
#                 else:
#                     factor = 2**(level-l)
#                     i_n,j_n = i//factor,j//factor
#                     print("its3", i_n,j_n)
                
#                 if (0 <= i_n < im.shape[0] and 0 <= j_n < im.shape[1]):
#                     print("yup iam exicuted")
#                     curr_sigma = sigma*(2**level)
#                     Laplacian = gaussian_laplace(im, sigma=curr_sigma, mode='constant')
#                     val_L = abs(((curr_sigma**2)*Laplacian[i_n,j_n]))
#                     response.append((val_L,level))
#             print("itsherenow")
#             max_response = max(response, key=lambda item: response[0])
#             __,charac_scale = max_response
#             eigenvalues_charac_scale = Harris_interest_points(image[charac_scale])
#             if l == 0:
#                 i_f,j_f = i,j
#             elif l>0:
#                 factor = 2**(l)
#                 i_f,j_f = i*factor,j*factor

#             min_eigval = np.min(eigenvalues_charac_scale[i_f,j_f])

#             if min_eigval < thresh: 
#                     print("here in loop1")
#                     interest_points.append((i_f,j_f,charac_scale))


#     return interest_points


# interest_points = scale_normalized_Laplacian(Pyramid_im1,20,0)

# print(interest_points)

In [ ]:
def load_image(path):
    img = cv2.imread(path)
    if img is None:
        img = plt.imread(path)
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def normalize_image(image):
    img_float = image.astype(np.float64)
    
    mean_val = np.mean(img_float)
    std_val = np.std(img_float)
    
    if std_val > 0:
        normalized = (img_float - mean_val) / std_val
    else:
        normalized = img_float - mean_val
    
    normalized = ((normalized - normalized.min()) / 
                  (normalized.max() - normalized.min()) * 255).astype(np.uint8)
    
    return normalized

img_1 = normalize_image(load_image("C:/Users/shalh/OneDrive/Desktop/Underwater Image/1.tif"))
img_2 = normalize_image(load_image("C:/Users/shalh/OneDrive/Desktop/Underwater Image/2.tif"))
img_3 = normalize_image(load_image("C:/Users/shalh/OneDrive/Desktop/Underwater Image/3.tif"))
img_4 = normalize_image(load_image("C:/Users/shalh/OneDrive/Desktop/Underwater Image/4.tif"))
img_5 = normalize_image(load_image("C:/Users/shalh/OneDrive/Desktop/Underwater Image/5.tif"))
img_6 = normalize_image(load_image("C:/Users/shalh/OneDrive/Desktop/Underwater Image/6.tif"))

sift_det =  cv2.SIFT.create()

fp1, desp1 = sift_det.detectAndCompute(img_1,None)
fp2, desp2 = sift_det.detectAndCompute(img_2, None)
fp3, desp3 = sift_det.detectAndCompute(img_3,None)
fp4, desp4 = sift_det.detectAndCompute(img_4, None)
fp5, desp5 = sift_det.detectAndCompute(img_5,None)
fp6, desp6 = sift_det.detectAndCompute(img_6, None)

FLANN_INDEX_KDTREE = 1
trees = dict(algorithm=FLANN_INDEX_KDTREE)
search = dict(checks=50)
flann = cv2.FlannBasedMatcher(trees,search)

matches12 = flann.knnMatch(desp1,desp2,k=2)
matches23 = flann.knnMatch(desp2,desp3,k=2)
matches34 = flann.knnMatch(desp3,desp4,k=2)
matches45 = flann.knnMatch(desp4,desp5,k=2)
matches56 = flann.knnMatch(desp5,desp6,k=2)




plt.figure(figsize=(15, 8))
plt.imshow(im12_matches, cmap='gray')
plt.title("matched points")
plt.axis('off')
plt.show()


In [ ]:
sorted_matches12 = []
sorted_matches23 = []
sorted_matches34 = []
sorted_matches45 = []
sorted_matches56 = []


matches = [matches12,matches23,matches34,matches45,matches56]

for i,j in matches12:
            if i.distance<0.7*j.distance:
                sorted_matches12.append(i)
for i,j in matches23:
            if i.distance<0.7*j.distance:
                sorted_matches23.append(i)
for i,j in matches34:
            if i.distance<0.7*j.distance:
                sorted_matches34.append(i)
for i,j in matches45:
            if i.distance<0.7*j.distance:
                sorted_matches45.append(i)
for i,j in matches56:
            if i.distance<0.7*j.distance:
                sorted_matches56.append(i)

im12_matches = cv2.drawMatches(img_1, fp1, img_2, fp2, sorted_matches12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im23_matches = cv2.drawMatches(img_2, fp2, img_3, fp3, sorted_matches23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im34_matches = cv2.drawMatches(img_3, fp3, img_4, fp4, sorted_matches34, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im45_matches = cv2.drawMatches(img_4, fp4, img_5, fp5, sorted_matches45, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im56_matches = cv2.drawMatches(img_5, fp5, img_6, fp6, sorted_matches56, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

matched12_xy = []
matched23_xy = []
matched34_xy = []
matched45_xy = []
matched56_xy = []
for match in sorted_matches12:
    pt1 = fp1[match.queryIdx].pt
    pt2 = fp2[match.trainIdx].pt
    matched12_xy.append((pt1, pt2))
for match in sorted_matches23:
    pt2 = fp2[match.queryIdx].pt
    pt3 = fp3[match.trainIdx].pt
    matched23_xy.append((pt2, pt3))
for match in sorted_matches34:
    pt3 = fp3[match.queryIdx].pt
    pt4 = fp4[match.trainIdx].pt
    matched34_xy.append((pt3, pt4))
for match in sorted_matches45:
    pt4 = fp4[match.queryIdx].pt
    pt5 = fp5[match.trainIdx].pt
    matched45_xy.append((pt4, pt5))
for match in sorted_matches56:
    pt5 = fp5[match.queryIdx].pt
    pt6 = fp6[match.trainIdx].pt
    matched56_xy.append((pt5, pt6))

im1_pt,im2_pt = matched12_xy[0]

print(np.float32([matched12_xy[i][1] for i,j in enumerate(matched12_xy)]))



In [ ]:



# # LMS algo for Homographies calculation 

# def LMS_homography_calc(pairs, trials=1000):
#     best_H = None
#     best_median_residual = np.inf
#     im1_pt_all = np.float32([pairs[i][0] for i,j in enumerate(pairs)]).reshape(-1, 1, 2)
#     im2_pt_all = np.float32([pairs[i][1] for i,j in enumerate(pairs)]).reshape(-1, 1, 2)

#     for trial in range(trials):
#         index = np.random.choice(len(pairs), 4, replace=False)
#         im1_pt = np.float32([pairs[i][0] for i in index])
#         im2_pt = np.float32([pairs[i][1] for i in index])

#         H = cv2.getPerspectiveTransform(im1_pt,im2_pt)
#         transformed_im1pt = cv2.perspectiveTransform(im1_pt_all, H)
#         residual_err = np.sqrt(np.sum((transformed_im1pt - im2_pt_all)**2, axis=2)).flatten()
#         median_residual = np.median(residual_err)

#         if median_residual < best_median_residual:
#             best_H = H
#             best_median_residual = median_residual

#     return best_H, best_median_residual

# H12, best_median_res12 = LMS_homography_calc(matched12_xy, trials=1000)
# H23, best_median_res23 = LMS_homography_calc(matched23_xy, trials=1000)
# H34, best_median_res34 = LMS_homography_calc(matched34_xy, trials=1000)
# H45, best_median_res45 = LMS_homography_calc(matched45_xy, trials=1000)
# H56, best_median_res56 = LMS_homography_calc(matched56_xy, trials=1000)

# print(H12)

In [ ]:
def pts(pairs):
    im1_pt_all = np.float32([pairs[i][0] for i,j in enumerate(pairs)]).reshape(-1, 1, 2)
    im2_pt_all = np.float32([pairs[i][1] for i,j in enumerate(pairs)]).reshape(-1, 1, 2)
    return im1_pt_all,im2_pt_all


src_pts12,dst_pts12 = pts(matched12_xy)
src_pts23,dst_pts23 = pts(matched23_xy)
src_pts34,dst_pts34 = pts(matched34_xy)
src_pts45,dst_pts45 = pts(matched45_xy)
src_pts56,dst_pts56 = pts(matched56_xy)



H12i, mask12 = cv2.findHomography(src_pts12, dst_pts12, cv2.RANSAC)
H23i, mask23 = cv2.findHomography(src_pts23, dst_pts23, cv2.RANSAC)
H34i, mask34 = cv2.findHomography(src_pts34, dst_pts34, cv2.RANSAC)
H45i, mask45 = cv2.findHomography(src_pts45, dst_pts45, cv2.RANSAC)
H56i, mask56 = cv2.findHomography(src_pts56, dst_pts56, cv2.RANSAC)
print("h12i", H12i)

inlier_src12, inlier_dst12 = src_pts12[mask12.ravel() == 1], dst_pts12[mask12.ravel() == 1]
inlier_src23, inlier_dst23 = src_pts23[mask23.ravel() == 1], dst_pts23[mask23.ravel() == 1]
inlier_src34, inlier_dst34 = src_pts34[mask34.ravel() == 1], dst_pts34[mask34.ravel() == 1]
inlier_src45, inlier_dst45 = src_pts45[mask45.ravel() == 1], dst_pts45[mask45.ravel() == 1]
inlier_src56, inlier_dst56 = src_pts56[mask56.ravel() == 1], dst_pts56[mask56.ravel() == 1]


def residual_function(h_params, src_pts, dst_pts):
    H = np.append(h_params, 1).reshape(3, 3)
    
    src_homo = np.column_stack([src_pts.reshape(-1, 2), np.ones(len(src_pts.reshape(-1, 2)))])
    projected_homo = (H @ src_homo.T).T
    projected_2d = projected_homo[:, :2] / projected_homo[:, 2:3]
    
    return (projected_2d - dst_pts.reshape(-1, 2)).flatten()

def LM(H_init, src_pts, dst_pts):
    h_params = H_init.flatten()[:8]
    
    result = least_squares(residual_function, h_params, 
                          args=(src_pts, dst_pts), 
                          method='lm')
    
    H_refined = np.append(result.x, 1).reshape(3, 3)
    return H_refined

H12 = LM(H12i, inlier_src12, inlier_dst12)
H23 = LM(H23i, inlier_src23, inlier_dst23)
H34 = LM(H34i, inlier_src34, inlier_dst34)
H45 = LM(H45i, inlier_src45, inlier_dst45)
H56 = LM(H56i, inlier_src56, inlier_dst56)
print(H12)





In [ ]:
def sequential_image_stitching(images,homographies):
    result = images[0].copy()
    canvas_w = images[0].shape[1]*3
    canvas_h = images[0].shape[0]*3

    canvas = np.zeros((canvas_h, canvas_w), dtype=np.uint8)

    center_transform = np.array([[1, 0, canvas_w//2 - images[0].shape[1]//2],
                                 [0, 1, canvas_h//2 - images[0].shape[0]//2],
                                 [0, 0, 1]], dtype=np.float32)
    
    # Place first image in center
    first_H = center_transform
    warped_first = cv2.warpPerspective(images[0],  first_H, (canvas_w, canvas_h))
    canvas = warped_first.copy()

    # x_offset = canvas_w//4
    # y_offset = canvas_h//4
    # canvas[y_offset:y_offset+images[0].shape[0], x_offset:x_offset+images[0].shape[1]] = images[0]

    cum_H = first_H 
    for i in range(1,len(images)):
        cum_H =  np.dot(cum_H, homographies[i-1])
        warped = cv2.warpPerspective(images[i], cum_H, (canvas_w, canvas_h))
        alpha = 0.5
        beta = 1.0 - alpha
        warped = cv2.addWeighted(warped_first, alpha, warped, beta, 0)
      
        mask = warped > 0
        canvas[mask] = warped[mask]
    return canvas

# mosaic = sequential_image_stitching([img_1, img_2, img_3], [H12,H23])

images = [img_1, img_2, img_3, img_4, img_5, img_6]
homographies = [H12, H23, H34, H45, H56]

for i in range(len(images)-1):
    mosaic = sequential_image_stitching([images[i], images[i+1]], [homographies[i]])
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic, cmap='gray')
    plt.title(f'Mosaic: Image {i+1} + Image {i+2}')
    plt.axis('off')
    plt.show()


In [ ]:
matches16 = flann.knnMatch(desp1,desp6,k=2)
matches25 = flann.knnMatch(desp2,desp5,k=2)
matches24 = flann.knnMatch(desp2,desp4,k=2)
matches35 = flann.knnMatch(desp3,desp5,k=2)
matches15 = flann.knnMatch(desp1,desp5,k=2)
matches26 = flann.knnMatch(desp2,desp6,k=2)

sorted_matches16 = []
sorted_matches25 = []
sorted_matches24 = []
sorted_matches35 = []
sorted_matches15 = []
sorted_matches26 = []


matches = [matches12,matches23,matches34,matches45,matches56]

for i,j in matches16:
            if i.distance<0.7*j.distance:
                sorted_matches16.append(i)
for i,j in matches25:
            if i.distance<0.7*j.distance:
                sorted_matches25.append(i)
for i,j in matches24:
            if i.distance<0.7*j.distance:
                sorted_matches24.append(i)
for i,j in matches35:
            if i.distance<0.7*j.distance:
                sorted_matches35.append(i)
for i,j in matches15:
            if i.distance<0.7*j.distance:
                sorted_matches15.append(i)
for i,j in matches16:
            if i.distance<0.7*j.distance:
                sorted_matches16.append(i)


im16_matches = cv2.drawMatches(img_1, fp1, img_6, fp6, sorted_matches16, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im25_matches = cv2.drawMatches(img_2, fp2, img_5, fp5, sorted_matches25, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im24_matches = cv2.drawMatches(img_2, fp2, img_4, fp4, sorted_matches24, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im35_matches = cv2.drawMatches(img_3, fp3, img_5, fp5, sorted_matches35, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im15_matches = cv2.drawMatches(img_1, fp1, img_5, fp5, sorted_matches15, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im26_matches = cv2.drawMatches(img_2, fp2, img_6, fp6, sorted_matches26, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)



matched16_xy = []
matched25_xy = []
matched24_xy = []
matched35_xy = []
matched15_xy = []
matched26_xy = []

for match in sorted_matches16:
    pt1 = fp1[match.queryIdx].pt
    pt6 = fp6[match.trainIdx].pt
    matched16_xy.append((pt1, pt6))
for match in sorted_matches25:
    pt2 = fp2[match.queryIdx].pt
    pt5 = fp5[match.trainIdx].pt
    matched25_xy.append((pt2, pt5))
for match in sorted_matches24:
    pt2 = fp2[match.queryIdx].pt
    pt4 = fp4[match.trainIdx].pt
    matched24_xy.append((pt2, pt4))
for match in sorted_matches35:
    pt3 = fp3[match.queryIdx].pt
    pt5 = fp5[match.trainIdx].pt
    matched35_xy.append((pt3, pt5))
for match in sorted_matches15:
    pt1 = fp1[match.queryIdx].pt
    pt5 = fp5[match.trainIdx].pt
    matched15_xy.append((pt1, pt5))
for match in sorted_matches26:
    pt2 = fp2[match.queryIdx].pt
    pt6 = fp6[match.trainIdx].pt
    matched26_xy.append((pt2, pt6))

#tried finding a match between image 2to5 and even from image 1 to 5 and 2 to 6 but didnt get any matches 


src_pts16,dst_pts16 = pts(matched16_xy)
src_pts25,dst_pts25 = pts(matched25_xy)
src_pts24,dst_pts24 = pts(matched24_xy)
src_pts35,dst_pts35 = pts(matched35_xy)
src_pts15,dst_pts15 = pts(matched15_xy)
src_pts26,dst_pts26 = pts(matched26_xy)

matched = [matched16_xy, matched25_xy, matched24_xy, matched35_xy, matched15_xy, matched26_xy]

# print(len(matched[1]))

if len(matched[0]) >= 4:
    H16, mask16 = cv2.findHomography(src_pts16, dst_pts16, cv2.RANSAC)
    mosaic = sequential_image_stitching([images[0], images[5]], [H16])
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic, cmap='gray')
    plt.title(f'Mosaic: Image 1 + Image 6')
    plt.axis('off')
    plt.show()
else:
    print("not enough points to really match 16")

if len(matched[1]) >= 4:
    H25, mask25 = cv2.findHomography(src_pts25, dst_pts25, cv2.RANSAC)
    mosaic = sequential_image_stitching([images[1], images[4]], [H25])
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic, cmap='gray')
    plt.title(f'Mosaic: Image 2 + Image 5')
    plt.axis('off')
    plt.show()
else:
    print("not enough points to really match 25")

if len(matched[2]) >= 4:
    H24, mask24 = cv2.findHomography(src_pts24, dst_pts24, cv2.RANSAC)
    mosaic = sequential_image_stitching([images[1], images[3]], [H24])
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic, cmap='gray')
    plt.title(f'Mosaic: Image {1+1} + Image {2+2}')
    plt.axis('off')
    plt.show()
else:
    print("not enough points to really match 24")

if len(matched[3]) >= 4:
    H35, mask35 = cv2.findHomography(src_pts35, dst_pts35, cv2.RANSAC)
    mosaic = sequential_image_stitching([images[2], images[4]], [H35])
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic, cmap='gray')
    plt.title(f'Mosaic: Image {2+1} + Image {3+2}')
    plt.axis('off')
    plt.show()
else:
    print("not enough points to really match 35")

if len(matched[4]) >= 4:
    H15, mask15 = cv2.findHomography(src_pts15, dst_pts15, cv2.RANSAC)
    mosaic = sequential_image_stitching([images[0], images[4]], [H15])
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic, cmap='gray')
    plt.title(f'Mosaic: Image {0+1} + Image {3+2}')
    plt.axis('off')
    plt.show()
else:
    print("not enough points to really match 15")

if len(matched[5]) >= 4:
    H26, mask26 = cv2.findHomography(src_pts26, dst_pts26, cv2.RANSAC)
    mosaic = sequential_image_stitching([images[1], images[5]], [H26])
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic, cmap='gray')
    plt.title(f'Mosaic: Image {1+1} + Image {4+2}')
    plt.axis('off')
    plt.show()
else:
    print("not enough points to really match 26")


